In [1]:
from sklearn.metrics import mean_absolute_percentage_error as mape
from all_functions import *
from darts import TimeSeries
import os
from darts.models import StatsForecastAutoETS
from darts.models import StatsForecastAutoARIMA
import statsmodels.api as sm
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
def percorrer_csv(diretorios, save_dir='./', horizon=5):
    cont = 0
    
    resultado_file = save_dir+'/result_ets.csv'
    train = pd.Series()
    for diretorio in diretorios.split():
        if os.path.isdir(diretorio):
            for root, _, arquivos in os.walk(diretorio):
                for arquivo in arquivos:
                    if arquivo.endswith('.csv'):
                        caminho_arquivo = os.path.join(root, arquivo)
                        try:
                            paths = caminho_arquivo.split('/')
                            file_csv = paths[-1]
                            estado = file_csv.split('_')[2]
                            derivado = file_csv.split('_')[3].replace('.csv', '')
                            cidade = file_csv.split('_')[1]
                            df = pd.read_csv(caminho_arquivo, sep=";")
                            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y')

                            df.set_index('timestamp', inplace=True)
                            start_timestamp = '1998-01-01'  # ou qualquer data inicial que você queira
                            index_series = pd.date_range(start=start_timestamp, periods=len(df), freq='Y')
                            # series = pd.Series(series_value, index=index_series)
                            serie_m3 = pd.Series(df['m3'].values, index=index_series)
                            train, test = train_test_stats(serie_m3, horizon)
                            w = len(train)
                            # train_tf, _, _ = rolling_window_series(train, horizon)
                            train_tf = znorm(train)
                            train_darts = TimeSeries.from_series(train_tf)
                            
                            if len(serie_m3) > 12:
                                test_index = test
                                #roda 5 vezes para 5 anos
                                preds_final = []
                                preds_series = pd.Series()
                                train_concat = train
                                # model = StatsForecastAutoETS(model='AAN')
                                # model = StatsForecastAutoARIMA(season_length=1)
                                model = ETSModel(train_tf, trend='add', damped_trend=True, seasonal_periods=1, freq='A').fit(full_output=False, disp=False)


                                predictions = model.forecast(steps=horizon)
                                preds_norm = pd.Series(predictions, index=test.index)
                                # model.fit(train_darts)
                                # result = model.predict(n=horizon)
                                # preds_norm = pd.Series(result.values().flatten().tolist(), index=test.index)
                                preds_real = reverse_transform_norm_preds(preds_norm, train_concat, "normal", w=horizon)
                                preds_final = preds_real.values
                                # for i in range(5):
                                #     model = StatsForecastAutoETS()
                                #     model.fit(train_darts)
                                #     result = model.predict(n=1)
                                #     new_index = test_index.index[0] + pd.DateOffset(years=i)
                                #     new_index = pd.DatetimeIndex([new_index])
                                #     preds_norm = pd.Series(result.values().flatten().tolist(), index=new_index)
                                #     w = len(train_concat)
                                #     preds_real = reverse_transform_norm_preds(preds_norm, train_concat, "normal", w=w)
                                #     train_concat = pd.concat([train_concat, preds_real])
                                #     preds_final.extend(preds_real.values)
                                #     if preds_series.empty:
                                #         preds_series = preds_real
                                #     else:
                                #         preds_series = pd.concat([preds_series, preds_real])
                                    
                                #     #concat train com preds_real
                                #     # train_concat_tf, _, _ = rolling_window_series(train_concat, w)
                                #     train_concat_tf = znorm(train_concat)
                                #     train_darts = TimeSeries.from_series(train_concat_tf)
                                    
                                cont+=1
                                mape_result = mape(test.values, preds_final)
                                pocid_result = pocid(test.values, preds_final)
                                dados = {
                                    'PRODUCT': derivado,
                                    'UF': estado.upper(),
                                    'MODEL': 'ETS',
                                    'PARAMETERS': str({'trend':'add', 'damped_trend':True, 'seasonal_periods':1, 'freq':'A'}),
                                    # 'mape': mape_result,
                                    # 'pocid': pocid_result,
                                    'city': cidade,
                                    'test': [list(test.values)],
                                    'preds': [list(preds_final)],
                                    # 'index_test': [test.index],
                                    # 'index_preds': [preds_series.index]
                                    
                                }
                                df_result = pd.DataFrame(dados)
                                df_result = df_result.reset_index(drop=True)
                                print(cont)

                                if not os.path.exists(resultado_file):
                                    df_result.to_csv(resultado_file, sep=';', index=False, mode='w', header=True)
                                else:
                                    df_result.to_csv(resultado_file, sep=';', index=False, mode='a', header=False)
                        except Exception as e:
                            print(f"Erro ao ler {arquivo}: {e}")
        else:
            print(train)
            print(f"O diretório {diretorio} não existe ou não é uma pasta válida.")




/home/user/miniconda3/envs/lucas/lib/python3.11/site-packages/optuna/study/_optimize.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [49]:
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:.4f}".format

model = "FT_catboost"
part = '/normal'
csv_files = [
    f"./timeseries/mestrado/resultados/{model}{part}/ANP_MONTHLY.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/AUSTRALIAN_ELECTRICITY_DEMAND_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/M4_HOURLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/M4_WEEKLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/NN5_DAILY_DATASET_WITHOUT_MISSING_VALUES.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/NN5_WEEKLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/PEDESTRIAN_COUNTS_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/US_BIRTHS_DATASET.csv",
]

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, sep=";")
        print(f"{csv_file.split('/')[-1]}: {len(df)} entries")
    except Exception as e:
        print(f"Erro ao ler {csv_file}: {e}")

ANP_MONTHLY.csv: 5642 entries
AUSTRALIAN_ELECTRICITY_DEMAND_DATASET.csv: 20 entries
M4_HOURLY_DATASET.csv: 1656 entries
M4_WEEKLY_DATASET.csv: 1436 entries
NN5_DAILY_DATASET_WITHOUT_MISSING_VALUES.csv: 444 entries
NN5_WEEKLY_DATASET.csv: 444 entries
PEDESTRIAN_COUNTS_DATASET.csv: 264 entries
US_BIRTHS_DATASET.csv: 4 entries


In [21]:
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:.4f}".format
df_agent = pd.read_csv(
    "./timeseries/mestrado/resultados/ADE/ANP_MONTHLY.csv",
    sep=";",
)
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
df_agent = df_agent[df_agent['final_test'] == '2024-11-30']

print(len(df_agent))
print(
    df_agent[["mape", "pocid", "smape", "rmse", "msmape", "mae"]]
    .apply(pd.to_numeric, errors="coerce")
    .mean()
)

182
mape        0.1688
pocid      67.6324
smape       0.1071
rmse     4455.8191
msmape      0.1071
mae      3761.5651
dtype: float64


In [22]:
df_agent = pd.read_csv(
    "./timeseries/mestrado/resultados/BEST_MEAN/ANP_MONTHLY.csv",
    sep=";",
)
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
df_agent = df_agent[df_agent['final_test'] == '2024-11-30']

print(len(df_agent))
print(
    df_agent[["mape", "pocid", "smape", "rmse", "msmape", "mae"]]
    .apply(pd.to_numeric, errors="coerce")
    .mean()
)

182
mape     1909639065279761.2500
pocid                  66.8332
smape                   0.2203
rmse                 5928.3099
msmape                  0.2201
mae                  5078.4335
dtype: float64


In [7]:
import numpy as np
predictions_val = {
    "THETA": [17866.08934355, 17144.80877427, 15254.82768456, 17103.34824538, 16019.72145045, 16809.28489874, 17700.12233432, 18537.68322061, 19161.68786003, 18545.03624378, 19271.48287791, 18913.88997639],
    "FT_catboost": [16413.09711353, 17307.99150984, 16641.3797852, 17893.33707602, 16333.22457947, 17197.48149865, 17136.21290436, 17786.45849121, 17253.56123353, 17112.93159883, 16913.48233282, 16626.96867976],
    "DWT_catboost": [18114.39598359, 17960.48813212, 17783.57486875, 17688.82118656, 17585.03278393, 17745.14227767, 17638.62737912, 17646.17618313, 17643.0873276, 17652.8525517, 17609.07633908, 17616.40332561],
}
predictions = {
    "THETA": [18345.6940208, 17874.10983595, 16222.97612274, 17834.21723458, 16755.47385476, 17504.31572072, 18362.58535134, 18904.91805159, 19501.2955687, 18887.74346103, 19515.8698591, 19162.3682469],
    "FT_catboost": [17729.70542317, 17367.36165659, 16797.76082268, 17016.07584365, 16779.51393948, 17312.56091945, 17543.29106472, 17704.00934383, 17732.31867146, 17519.12232138, 17485.72105834, 17363.68890376],
    "DWT_catboost": [17842.93000058, 17500.06127439, 16702.69910431, 16955.12667547, 16820.62850256, 17339.22385666, 17528.69917841, 17642.78761942, 17726.55757098, 17571.69823637, 17548.1240138, 17409.61835574],
}

all_preds = list(predictions.values())
np.mean(all_preds, axis=0)

array([17972.77648152, 17580.51092231, 16574.47868324, 17268.47325123,
       16785.20543227, 17385.36683228, 17811.52519816, 18083.90500495,
       18320.05727038, 17992.85467293, 18183.23831041, 17978.55850213])

In [ ]:
validation_test = [17656.623, 19507.078, 15680.762, 19775.546, 13736.136, 17221.028, 18352.012, 20327.377, 21175.424, 18516.637, 19864.665, 18523.176]

validation_predictions = {
    "ARIMA": [20189.34563269, 20192.85789627, 20092.93080571, 20125.30269528, 19897.04910318, 20355.62201653, 21603.84938594, 20580.46644655, 21348.37952553, 20451.24241379, 20830.78699143, 20366.88668088],
"ETS": [18062.62317529, 17393.33066554, 15407.84271242, 17092.38074023, 16162.45141857, 16844.88650947, 17782.62518231, 18540.99028846, 19297.52887647, 18610.390685, 19436.7535238, 19007.89060085],
"THETA": [17866.08934355, 17144.80877427, 15254.82768456, 17103.34824538, 16019.72145045, 16809.28489874, 17700.12233432, 18537.68322061, 19161.68786003, 18545.03624378, 19271.48287791, 18913.88997639],
"svr": [18759.42203864, 18759.42210619, 18759.4221062, 18759.4221062, 18759.42210639, 18759.42210538, 18759.42213586, 18759.4221062, 18759.4221062, 18759.4221062, 18759.4221062, 18759.4221062],
"rf": [17103.19423397, 17803.67250428, 17731.06974653, 18104.01435002, 18000.64727707, 18303.83074421, 18103.89545828, 18113.67717747, 18026.40149462, 18120.85642935, 17798.38598629, 17688.20732412],
"catboost": [16997.71257262, 18223.88166564, 17910.64906155, 18052.06112336, 17395.51510697, 17464.68223325, 16751.90487253, 15703.21729542, 15213.6946995, 15455.24530211, 16404.94868817, 18586.2942714],
"CWT_svr": [18759.51542668, 18515.20822027, 18295.9953686, 18150.54620463, 18073.18622798, 18188.19369318, 18151.85355396, 18048.70977966, 17975.13513241, 18000.12467132, 17998.57014806, 17976.45072006],
"DWT_svr": [18759.51542668, 18515.20822027, 18295.9953686, 18150.54620463, 18073.18622798, 18188.19369318, 18151.85355396, 18048.70977966, 17975.13513241, 18000.12467132, 17998.57014806, 17976.45072006],
"FT_svr": [16986.4041512, 18005.59142402, 18099.29958901, 18571.83507727, 18548.55281724, 18406.59623913, 18653.85029216, 18370.60372238, 17420.79279064, 17460.84211587, 17347.69927974, 17141.04602255],
"CWT_rf": [16810.50631223, 16970.34844394, 16968.19324107, 16680.46123042, 16999.2317529, 16553.27749375, 16781.29620828, 16431.03309906, 16644.58693919, 16396.60590851, 16386.40946308, 16481.18908182],
"DWT_rf": [16726.88958047, 17370.9128624, 16684.84138993, 17269.19743572, 17246.71066812, 17060.49243543, 17086.71343359, 16896.54214381, 16474.99586124, 16581.70867416, 16356.79727089, 16483.25983542],
"FT_rf": [15737.96054126, 16124.04505268, 17410.61249198, 17342.36712357, 17898.41309604, 18194.83951578, 18306.64199349, 18295.73810922, 17997.28435946, 17762.59600298, 16908.27926607, 16875.18971294],
"CWT_catboost": [18021.92606255, 17817.83554492, 18011.85713323, 17890.44975533, 17949.9677973, 18120.2264075, 18115.2101686, 18060.46296962, 17863.39983706, 17781.64505428, 17662.88374363, 17749.57771547],
"DWT_catboost": [18114.39598359, 17960.48813212, 17783.57486875, 17688.82118656, 17585.03278393, 17745.14227767, 17638.62737912, 17646.17618313, 17643.0873276, 17652.8525517, 17609.07633908, 17616.40332561],
"FT_catboost": [16413.09711353, 17307.99150984, 16641.3797852, 17893.33707602, 16333.22457947, 17197.48149865, 17136.21290436, 17786.45849121, 17253.56123353, 17112.93159883, 16913.48233282, 16626.96867976]
}